In [21]:
from typing import List, TypedDict, Annotated, Literal
from dotenv import load_dotenv
import operator
import json
import os

from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from pydantic import BaseModel
from langchain.schema import HumanMessage, AIMessage

from src.nodes import *
from src.state import State
from src.util import next_node

In [23]:
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")
# print(os.environ["GOOGLE_API_KEY"])

In [26]:
gemini = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", temperature=0.7)
evaluator = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", temperature=0.3)

In [28]:
# Config
max_iterations = 5

In [29]:
builder = StateGraph(State)

nodes = {
    "Supervisor": Supervisor(),
    "Agent1": Agent("Agent 1 - Impact", gemini),
    "Evaluator": Evaluator(evaluator),
    "loop_control": LoopControlNode(max_iterations),
}

In [30]:
for name, node in nodes.items():
    builder.add_node(name, node)

In [ ]:
# Define the flow
builder.add_edge(START, "supervisor")

# Edges from Supervisor to Agents
builder.add_edge("Supervisor", "Agent1")

# Edges from Agents to Reviewer
builder.add_edge("Agent1", "Evaluator")

# From Reviewer to Loop Control Node
builder.add_edge("Evaluator", "loop_control")